# অটোজেন টুল ব্যবহারের উদাহরণ


## প্রয়োজনীয় প্যাকেজগুলো আমদানি করুন


In [ ]:
import os
import json

import requests
from autogen_agentchat.agents import AssistantAgent
from autogen_core.models import UserMessage
from autogen_ext.models.azure import AzureAIChatCompletionClient
from azure.core.credentials import AzureKeyCredential
from autogen_core import CancellationToken
from autogen_core.tools import FunctionTool
from autogen_agentchat.messages import TextMessage
from autogen_agentchat.ui import Console
from typing import Any, Callable, Set, Dict, List, Optional

## ক্লায়েন্ট তৈরি করা

এই উদাহরণে, আমরা [GitHub Models](https://aka.ms/ai-agents-beginners/github-models) ব্যবহার করব LLM-এ অ্যাক্সেস পাওয়ার জন্য।

`model` হিসেবে `gpt-4o-mini` নির্ধারণ করা হয়েছে। GitHub Models মার্কেটপ্লেসে উপলব্ধ অন্য কোনো মডেলে পরিবর্তন করে দেখুন বিভিন্ন ফলাফল কেমন হয়।

একটি দ্রুত পরীক্ষা হিসেবে, আমরা একটি সহজ প্রম্পট চালাব - `What is the capital of France`।


In [ ]:
client = AzureAIChatCompletionClient(
    model="gpt-4o-mini",
    endpoint="https://models.inference.ai.azure.com",
    # To authenticate with the model you will need to generate a personal access token (PAT) in your GitHub settings.
    # Create your PAT token by following instructions here: https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens
    credential=AzureKeyCredential(os.environ["GITHUB_TOKEN"]),
    model_info={
        "json_output": True,
        "function_calling": True,
        "vision": True,
        "family": "unknown",
    },
)

result = await client.create([UserMessage(content="What is the capital of France?", source="user")])
print(result)

## ফাংশন সংজ্ঞায়িত করা

এই উদাহরণে, আমরা এজেন্টকে একটি টুলের অ্যাক্সেস দেব যা একটি ফাংশন, যেখানে উপলব্ধ ছুটির গন্তব্যগুলোর তালিকা এবং তাদের প্রাপ্যতা থাকবে।

আপনি ভাবতে পারেন যে এটি এমন একটি পরিস্থিতি হতে পারে যেখানে একটি ট্রাভেল এজেন্টের কাছে উদাহরণস্বরূপ একটি ট্রাভেল ডাটাবেসের অ্যাক্সেস থাকতে পারে।

এই নমুনাটি অনুসরণ করার সময়, নতুন ফাংশন এবং টুল সংজ্ঞায়িত করার চেষ্টা করতে পারেন যা এজেন্ট কল করতে পারে।


In [ ]:
from typing import Dict, List, Optional


def vacation_destinations(city: str) -> tuple[str, str]:
    """
    Checks if a specific vacation destination is available
    
    Args:
        city (str): Name of the city to check
        
    Returns:
        tuple: Contains city name and availability status ('Available' or 'Unavailable')
    """
    destinations = {
        "Barcelona": "Available",
        "Tokyo": "Unavailable",
        "Cape Town": "Available",
        "Vancouver": "Available",
        "Dubai": "Unavailable",
    }

    if city in destinations:
        return city, destinations[city]
    else:
        return city, "City not found"

# Example usage:
# city, status = vacation_destinations("Barcelona")
# print(f"How about visiting {city}? It's currently {status} there!")

## ফাংশন টুল সংজ্ঞায়িত করা  
`vacation_destinations`-কে একটি `FunctionTool` হিসেবে ব্যবহার করার জন্য, আমাদের এটি একটি টুল হিসেবে সংজ্ঞায়িত করতে হবে।  

আমরা টুলটির একটি বিবরণও প্রদান করব, যা এজেন্টকে সাহায্য করবে ব্যবহারকারীর অনুরোধকৃত কাজের সাথে সম্পর্কিতভাবে টুলটি কী উদ্দেশ্যে ব্যবহৃত হয় তা সনাক্ত করতে।  


In [ ]:
get_vacations = FunctionTool(
    vacation_destinations, description="Search for vacation destinations and if they are available or not."
)

## এজেন্ট সংজ্ঞায়িত করা

এখন আমরা নিচের কোডে এজেন্ট তৈরি করতে পারি। আমরা `system_message` সংজ্ঞায়িত করি যাতে এজেন্ট ব্যবহারকারীদের ছুটির গন্তব্য খুঁজে পেতে সাহায্য করার নির্দেশনা পায়।

আমরা `reflect_on_tool_use` প্যারামিটারটিও true হিসেবে সেট করি। এটি LLM-কে টুল কলের প্রতিক্রিয়া গ্রহণ করে তা প্রাকৃতিক ভাষায় পাঠানোর সুযোগ দেয়।

আপনি পার্থক্য দেখতে চাইলে প্যারামিটারটি false হিসেবে সেট করতে পারেন।


In [ ]:
agent = AssistantAgent(
    name="assistant",
    model_client=client,
    tools=[get_vacations],
    system_message="You are a travel agent that helps users find vacation destinations.",
    reflect_on_tool_use=True,
)

## এজেন্ট চালানো

এখন আমরা এজেন্ট চালাতে পারি, যেখানে প্রাথমিক ব্যবহারকারীর বার্তাটি টোকিওতে ভ্রমণের অনুরোধ জানাচ্ছে।

আপনি এই শহরের গন্তব্য পরিবর্তন করতে পারেন, যাতে এজেন্ট শহরের প্রাপ্যতা সম্পর্কে কীভাবে প্রতিক্রিয়া জানায় তা দেখতে পারেন।


In [ ]:
async def assistant_run() -> None:
    response = await agent.on_messages(
        [TextMessage(content="I would like to take a trip to Tokyo", source="user")],
        cancellation_token=CancellationToken(),
    )
    print(response.inner_messages)
    print(response.chat_message)


# Use asyncio.run(assistant_run()) when running in a script.
await assistant_run()


---

**অস্বীকৃতি**:  
এই নথিটি AI অনুবাদ পরিষেবা [Co-op Translator](https://github.com/Azure/co-op-translator) ব্যবহার করে অনুবাদ করা হয়েছে। আমরা যথাসম্ভব সঠিক অনুবাদের চেষ্টা করি, তবে অনুগ্রহ করে মনে রাখবেন যে স্বয়ংক্রিয় অনুবাদে ত্রুটি বা অসঙ্গতি থাকতে পারে। নথিটির মূল ভাষায় লেখা সংস্করণটিকেই প্রামাণিক উৎস হিসেবে বিবেচনা করা উচিত। গুরুত্বপূর্ণ তথ্যের জন্য, পেশাদার মানব অনুবাদ ব্যবহার করার পরামর্শ দেওয়া হয়। এই অনুবাদ ব্যবহারের ফলে সৃষ্ট কোনো ভুল বোঝাবুঝি বা ভুল ব্যাখ্যার জন্য আমরা দায়ী নই।
